## Imports

In [1]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype

## Functions

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/Research/Functions/')

In [3]:
import a_DataHelpers
import b_AlgoFunctions
import c_PredEval

In [50]:
from a_DataHelpers import getPivotMonths,Time_DataSplit,TestTrain_DataSplit,SingleRatingMatrix
from a_DataHelpers import AllRatingMatrices,All_SingleStepRatMat,psiStep_RatMat,psiAllStep_RatMat
from a_DataHelpers import Find_NewUsersItems,adjustedPSI_DF,adjustedAllDF,get_NEWHoldout

from b_AlgoFunctions import integrator,getStartingValues,integratorOnMat,last_psiTrainMat
from b_AlgoFunctions import Updt_RowMatrix,getRow_Mat,row_update, Updt_ColMatrix,getCol_Mat,colunm_update
from b_AlgoFunctions import UsersItems_RatPair,getRowCol_psiupdt,ITEMS_defferredStatus,USERS_defferredStatus
from b_AlgoFunctions import V_listUpdate,SingleStep_UPDATE,ALLSTEPs_update

from c_PredEval import TopNPred,TQDMgetALLTopNPred, Hitrate_Eval,getAll_HitRate
from c_PredEval import TQDMgetALLTopNPred_ALLUSERS,getALLTopNPred_ALLUSERS
from c_PredEval import get_ALLRandPred,getAll_RandomHitRate,getMOSTPOP_Pred,getAll_MostPOPHitRate,getAll_MOSTPOP_Pred
from c_PredEval import topN_Index,get_ALLRandPred_2,getAll_RandomHitRate_2,getAll_MostPOPHitRate2

## Apply Functions on *AmazonBeauty*

###Step 1: Data Cleaning

In [ ]:
BeautyDFF = BeautyDFF.drop_duplicates()
BeautyDFF.shape

In [ ]:
sample_df = BeautyDFF.groupby(['userId_int']).agg({'productId_int':'count'}) 
sample_df = sample_df.rename(columns = {'productId_int':'Product_Count'}).reset_index() 
sample_ = sample_df[sample_df['Product_Count'] > 1]         #get list of users that have rated more than 5 products
print(sample_.shape[0])
onetime_userslist =  sample_['userId_int'].to_list()
#print(len(onetime_userslist))
#onetime_userslist[:10]

In [ ]:
# df_SAMPLE = BeautyDFF.groupby(['productId_int']).agg({'userId_int':'count'}) 
# df_SAMPLE = df_SAMPLE.rename(columns = {'userId_int':'User_Count'}).reset_index() 
# DF_Items = df_SAMPLE[df_SAMPLE['User_Count'] > 1]         #get list of users that have rated more than 5 products
# print(DF_Items.shape[0])
# onetime_itemslist =  DF_Items['productId_int'].to_list()
# print(len(onetime_itemslist))
# #onetime_itemslist[:10]

In [ ]:
DFF_ = BeautyDFF.loc[ BeautyDFF['userId_int'].isin(onetime_userslist) ]#& BeautyDFF['productId_int'].isin(onetime_itemslist)]
print(DFF_['productId_int'].nunique())
print(DFF_['userId_int'].nunique())

In [ ]:
#AMZB_1 = BeautyDFF.loc[BeautyDFF['userId_int'].isin(onetime_userslist) & BeautyDFF['productId_int'].isin(onetime_userslist)]
AMZB_1 = DFF_[['productId_int','userId_int','rating','timestamp']]
AMZB_1

In [ ]:
#AMZB_df = AMZB_11.rename(columns={'userId':'userId_int','productId':'productId_int'})
#AMZB_1['timestamp'] = pd.to_datetime(AMZB_1['timestamp']) 
AMZB_df = AMZB_1.copy()
print(AMZB_df.dtypes)
print()
print("nUnique users: ",AMZB_df['userId_int'].nunique())
print("nUnique items: ",AMZB_df['productId_int'].nunique())
AMZB_df = AMZB_df.sort_values(['timestamp'])
AMZB_df

In [ ]:
unique_USER_id = pd.unique(AMZB_df.userId_int)
unique_PRODUCT_id = pd.unique(AMZB_df.productId_int)

zipped_USER_id_pair = zip(unique_USER_id, range(0, len(unique_USER_id)+1, 1))
zipped_PRODUCT_id_pair = zip(unique_PRODUCT_id, range(0, len(unique_PRODUCT_id)+1, 1))

USER_id_dict = OrderedDict(zipped_USER_id_pair)
PRODUCT_id_dict = OrderedDict(zipped_PRODUCT_id_pair)

AMZB_df['userId'] = AMZB_df.userId_int.map(USER_id_dict)
AMZB_df['productId'] = AMZB_df.productId_int.map(PRODUCT_id_dict)
print(AMZB_df['userId'].max())
print(AMZB_df['productId'].max())
print()
print(AMZB_df['userId'].nunique())
print(AMZB_df['productId'].nunique())
AMZB_df

In [ ]:
AMZB_DF = AMZB_df[['userId','productId','rating','timestamp']]
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())
AMZB_DF

###Step 2: Import Clean Data 

In [5]:
AMZB_DF  = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Research/Test_Data/usedAMZB.csv')
AMZB_DF['timestamp'] = pd.to_datetime(AMZB_DF['timestamp']) 
print(AMZB_DF.shape)     #	productId_int	userId_int
print(AMZB_DF.dtypes)
AMZB_DF

(51869, 4)
userId                int64
productId             int64
rating                int64
timestamp    datetime64[ns]
dtype: object


,userId,productId,rating,timestamp
0,0,0,1,2001-06-08
1,1,0,1,2002-06-12
2,1,1,1,2002-06-12
3,2,1,1,2002-06-17
4,3,1,1,2002-07-04
...,...,...,...,...
51864,21470,6687,1,2018-09-21
51865,16839,5635,1,2018-09-21
51866,15319,8618,1,2018-09-25
51867,12925,5977,1,2018-09-25


In [47]:
print(AMZB_DF.shape)
print("For users: ")
print(AMZB_DF['userId'].nunique())
print(AMZB_DF['userId'].max())
print()
print("For items: ")
print(AMZB_DF['productId'].nunique())
print(AMZB_DF['productId'].max())
print()
print("Timestamp")
print(AMZB_DF['timestamp'].min())
print(AMZB_DF['timestamp'].max())

(51869, 4)
For users: 
22799
22798

For items: 
8620
8619

Timestamp
2001-06-08 00:00:00
2018-09-26 00:00:00


###Step 1: Data Splitting

In [6]:
pivotMonths_list  = getPivotMonths(AMZB_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-08-26 00:00:00'),
 Timestamp('2018-07-26 00:00:00'),
 Timestamp('2018-06-26 00:00:00'),
 Timestamp('2018-05-26 00:00:00'),
 Timestamp('2018-04-26 00:00:00'),
 Timestamp('2018-03-26 00:00:00'),
 Timestamp('2018-02-26 00:00:00'),
 Timestamp('2018-01-26 00:00:00'),
 Timestamp('2017-12-26 00:00:00'),
 Timestamp('2017-11-26 00:00:00'),
 Timestamp('2017-10-26 00:00:00'),
 Timestamp('2017-09-26 00:00:00'),
 Timestamp('2017-08-26 00:00:00'),
 Timestamp('2017-07-26 00:00:00'),
 Timestamp('2017-06-26 00:00:00'),
 Timestamp('2017-05-26 00:00:00'),
 Timestamp('2017-04-26 00:00:00'),
 Timestamp('2017-03-26 00:00:00')]

In [7]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZB_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-03-25 00:00:00
Num ΔA_train : 10


In [8]:
AllDF_list, PSITest_list, HOLDOUT_list, UserItemDF_list = TestTrain_DataSplit(AMZB_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [9]:
print(AllDF_list[0].shape)
print(UserItemDF_list[0].shape)
print(HOLDOUT_list[0].shape)

(50377, 4)
(50119, 4)
(258, 4)


### Step 2: Find new users and items

In [10]:
Tstps = AMZB_DF['timestamp']
tr_ts = ΔA_train[-1]['timestamp'].max()
print("ΔA_train_last max date: ",tr_ts)
AllDF_start = AMZB_DF.loc[Tstps <= tr_ts] 
print(AllDF_start['timestamp'].min())
print(AllDF_start['timestamp'].max())
print()
print(AllDF_list[0]['timestamp'].min())
print(AllDF_list[0]['timestamp'].max())

ΔA_train_last max date:  2018-01-25 00:00:00
2001-06-08 00:00:00
2018-01-25 00:00:00

2001-06-08 00:00:00
2018-02-25 00:00:00


In [11]:
New_usersList,New_itemsList = Find_NewUsersItems(AllDF_start,AllDF_list,'userId','productId',N_steps=8)
New_usersList[0]   #

array([22340, 22341, 22342, 22343, 22344, 22345, 22346, 22347, 22348,
       22349, 22350, 22351, 22352, 22353, 22354, 22355, 22356, 22357,
       22358, 22359, 22360, 22361, 22362, 22363, 22364, 22365, 22366,
       22367, 22368, 22369, 22370, 22371, 22372, 22373, 22374, 22375,
       22376, 22377, 22378, 22379, 22380, 22381, 22382, 22383, 22384,
       22385, 22386, 22387, 22388, 22389, 22390, 22391, 22392, 22393,
       22394, 22395, 22396, 22397, 22398, 22399, 22400, 22401, 22402,
       22403, 22404, 22405, 22406, 22407, 22408, 22409, 22410, 22411,
       22412, 22413, 22414, 22415, 22416, 22417, 22418, 22419, 22420,
       22421, 22422, 22423, 22424, 22425, 22426, 22427, 22428, 22429,
       22430, 22431, 22432, 22433, 22434, 22435, 22436, 22437, 22438,
       22439, 22440, 22441, 22442, 22443, 22444, 22445, 22446, 22447,
       22448, 22449, 22450])

In [12]:
New_itemsList[0]   

array([8328, 8329, 8330, 8331, 8332, 8333, 8334, 8335, 8336, 8337, 8338,
       8339, 8340, 8341, 8342, 8343, 8344, 8345, 8346, 8347, 8348, 8349,
       8350, 8351, 8352, 8353, 8354, 8355, 8356, 8357, 8358, 8359, 8360,
       8361, 8362, 8363, 8364, 8365, 8366, 8367, 8368, 8369, 8370, 8371,
       8372, 8373, 8374, 8375, 8376, 8377, 8378, 8379, 8380, 8381, 8382,
       8383, 8384, 8385, 8386, 8387, 8388, 8389, 8390, 8391, 8392, 8393,
       8394, 8395, 8396, 8397, 8398])

In [13]:
for i,j in zip(New_usersList,New_itemsList):
    print(i.size)
    print(j.size)
    print()

111
71

97
56

119
44

78
36

24
26

17
28

12
20

1
11



###Get Updt RatMat

In [16]:
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())   #

22340 8328


In [14]:
rows_i = AllDF_start['userId'].nunique()
cols_i = AllDF_start['productId'].nunique()
print(A0_df.shape)
A0_RatMat_updt = SingleRatingMatrix(A0_df,'userId', 'productId',rows_i,cols_i)
A0_RatMat_updt

(45139, 4)


<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
	with 44740 stored elements in Compressed Sparse Row format>

In [15]:
PSI_TrainMat_updt = AllRatingMatrices(ΔA_train,'userId','productId',rows_i ,cols_i)
PSI_TrainMat_updt

[<22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 553 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 518 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 432 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 397 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 1017 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 482 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse matrix of type '<class 'numpy.float64'>'
 	with 357 stored elements in Compressed Sparse Row format>,
 <22340x8328 sparse mat

In [16]:
print(ΔA_train[-1]['userId'].max(),AllDF_start['userId'].nunique())
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique()) 

22339 22340
22340 8328


### Allsteps Update

In [17]:
rows_i = AllDF_start['userId'].nunique()
cols_i = AllDF_start['productId'].nunique()
print(AllDF_start.shape)
AllDF_startRatMat = SingleRatingMatrix(AllDF_start,'userId', 'productId',rows_i,cols_i)

U_start,S_start,V_start = svds(AllDF_startRatMat,k=50)
V_start = V_start.T
print()
print("U_start shape: ",U_start.shape)
print("S_start shape: ",S_start.shape)
print("V_start shape: ",V_start.shape)

(50009, 4)

U_start shape:  (22340, 50)
S_start shape:  (50,)
V_start shape:  (8328, 50)


In [82]:
# U_start,S_start,V_start = last_psiTrainMat(A0_RatMat_updt,PSI_TrainMat_updt,k=50)
# print()
# print("U_start shape: ",U_start.shape)
# print("S_start shape: ",S_start.shape)
# print("V_start shape: ",V_start.shape)

100%|██████████| 10/10 [00:01<00:00,  8.75it/s]


U_start shape:  (22340, 50)
S_start shape:  (50, 50)
V_start shape:  (8328, 50)


In [18]:
U_list,S_list,V_list = [],[],[]
U_list.append(U_start)
S_list.append(S_start)
V_list.append(V_start)
print(len(U_list),len(V_list))

1 1


In [19]:
DItems_, DUsers_,In_DomainUSERS,In_DomainITEMS,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,U_list,S_list,V_list = ALLSTEPs_update(AllDF_start,
                                                          UserItemDF_list,New_itemsList,New_usersList,U_list,S_list,V_list,Nsteps=8,k=50,Forced_Orth=False)

  0%|          | 0/111 [00:00<?, ?it/s]

1st Updating Phase..

Is_UserCold? : False
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : Tr

  5%|▍         | 5/111 [00:00<00:34,  3.09it/s]


Is_UserCold? : False

Is_UserCold? : False

Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : False

Is_UserCold? : False

Is_UserCold? : True
Is_UserCold? : False

Is_UserCold? : False

Is_UserCold? : False


 12%|█▏        | 13/111 [00:00<00:17,  5.74it/s]

...


Is_ItemCold? : False

Is_ItemCold? : False

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True



100%|██████████| 78/78 [00:01<00:00, 47.09it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
2nd Updating Phase..

Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : True
Is_UserCold? : Tru


100%|██████████| 7/7 [00:31<00:00,  4.52s/it]

Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True
Is_ItemCold? : True


#### Output Check

In [20]:
print(len(In_DomainITEMS[8327+1:]))
print(In_DomainITEMS[8327+1:])
print()
print(len(In_DomainUSERS[22339+1:]))
print(In_DomainUSERS[22339+1:])

236
[8381, 8448, 8449, 8329, 8330, 8331, 8332, 8333, 8336, 8343, 8344, 8345, 8346, 8348, 8349, 8352, 8355, 8361, 8362, 8363, 8364, 8365, 8368, 8369, 8370, 8371, 8372, 8373, 8374, 8375, 8378, 8379, 8380, 8385, 8386, 8388, 8389, 8391, 8398, 8435, 8328, 8334, 8340, 8353, 8357, 8367, 8376, 8377, 8383, 8384, 8390, 8392, 8395, 8399, 8400, 8401, 8404, 8405, 8406, 8407, 8408, 8410, 8413, 8414, 8415, 8416, 8417, 8418, 8419, 8422, 8423, 8424, 8427, 8428, 8431, 8432, 8433, 8434, 8437, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 8450, 8451, 8452, 8453, 8454, 8468, 8409, 8411, 8412, 8420, 8436, 8360, 8506, 8517, 8455, 8456, 8457, 8458, 8459, 8461, 8464, 8465, 8467, 8469, 8470, 8471, 8472, 8474, 8475, 8476, 8478, 8479, 8481, 8482, 8483, 8484, 8487, 8489, 8490, 8495, 8351, 8507, 8463, 8421, 8350, 8499, 8500, 8501, 8504, 8505, 8508, 8509, 8510, 8511, 8512, 8513, 8514, 8515, 8516, 8518, 8520, 8521, 8522, 8523, 8524, 8530, 8531, 8532, 8354, 8519, 8525, 8526, 8529, 8460, 8498, 8356, 8535,

In [24]:
#deferred items and users
print(len(DItems_))
print(DItems_)
print()
print(len(DUsers_))
print(DUsers_)

56
[8609, 8610, 8612, 8614, 8615, 8616, 8617, 8618, 8619, 8566, 8567, 8570, 8572, 8573, 8585, 8586, 8546, 8547, 8502, 8503, 8527, 8528, 8533, 8534, 8466, 8473, 8477, 8480, 8485, 8486, 8488, 8491, 8492, 8493, 8494, 8496, 8497, 8402, 8403, 8425, 8426, 8429, 8430, 8335, 8338, 8339, 8341, 8342, 8347, 8358, 8359, 8382, 8393, 8394, 8396, 8397]

23
[22771, 22776, 22785, 22751, 22677, 22734, 22742, 22613, 22637, 22646, 22651, 22653, 22659, 22453, 22487, 22497, 22355, 22362, 22365, 22372, 22388, 22443, 22446]


In [25]:
for x in list(itemID_dict)[8355:]:
    print ("Id: {}, UpdtId: {} ".format(x,  itemID_dict[x]))   ##id ==key || updtedid == values  ,itemID_dict

Id: 8373, UpdtId: 8355 
Id: 8374, UpdtId: 8356 
Id: 8375, UpdtId: 8357 
Id: 8378, UpdtId: 8358 
Id: 8379, UpdtId: 8359 
Id: 8380, UpdtId: 8360 
Id: 8385, UpdtId: 8361 
Id: 8386, UpdtId: 8362 
Id: 8388, UpdtId: 8363 
Id: 8389, UpdtId: 8364 
Id: 8391, UpdtId: 8365 
Id: 8398, UpdtId: 8366 
Id: 8435, UpdtId: 8367 
Id: 8328, UpdtId: 8368 
Id: 8334, UpdtId: 8369 
Id: 8340, UpdtId: 8370 
Id: 8353, UpdtId: 8371 
Id: 8357, UpdtId: 8372 
Id: 8367, UpdtId: 8373 
Id: 8376, UpdtId: 8374 
Id: 8377, UpdtId: 8375 
Id: 8383, UpdtId: 8376 
Id: 8384, UpdtId: 8377 
Id: 8390, UpdtId: 8378 
Id: 8392, UpdtId: 8379 
Id: 8395, UpdtId: 8380 
Id: 8399, UpdtId: 8381 
Id: 8400, UpdtId: 8382 
Id: 8401, UpdtId: 8383 
Id: 8404, UpdtId: 8384 
Id: 8405, UpdtId: 8385 
Id: 8406, UpdtId: 8386 
Id: 8407, UpdtId: 8387 
Id: 8408, UpdtId: 8388 
Id: 8410, UpdtId: 8389 
Id: 8413, UpdtId: 8390 
Id: 8414, UpdtId: 8391 
Id: 8415, UpdtId: 8392 
Id: 8416, UpdtId: 8393 
Id: 8417, UpdtId: 8394 
Id: 8418, UpdtId: 8395 
Id: 8419, UpdtId

In [26]:
for x in list(userID_dict)[22339:]:
    print ("Id: {}, UpdtId: {} ".format(x,  userID_dict[x]))   ##id ==key || updtedid == values 

Id: 22339, UpdtId: 22339 
Id: 22340, UpdtId: 22340 
Id: 22341, UpdtId: 22341 
Id: 22342, UpdtId: 22342 
Id: 22345, UpdtId: 22343 
Id: 22346, UpdtId: 22344 
Id: 22348, UpdtId: 22345 
Id: 22349, UpdtId: 22346 
Id: 22350, UpdtId: 22347 
Id: 22351, UpdtId: 22348 
Id: 22352, UpdtId: 22349 
Id: 22353, UpdtId: 22350 
Id: 22356, UpdtId: 22351 
Id: 22357, UpdtId: 22352 
Id: 22358, UpdtId: 22353 
Id: 22361, UpdtId: 22354 
Id: 22364, UpdtId: 22355 
Id: 22366, UpdtId: 22356 
Id: 22371, UpdtId: 22357 
Id: 22373, UpdtId: 22358 
Id: 22375, UpdtId: 22359 
Id: 22376, UpdtId: 22360 
Id: 22378, UpdtId: 22361 
Id: 22382, UpdtId: 22362 
Id: 22383, UpdtId: 22363 
Id: 22385, UpdtId: 22364 
Id: 22387, UpdtId: 22365 
Id: 22390, UpdtId: 22366 
Id: 22391, UpdtId: 22367 
Id: 22392, UpdtId: 22368 
Id: 22393, UpdtId: 22369 
Id: 22394, UpdtId: 22370 
Id: 22395, UpdtId: 22371 
Id: 22396, UpdtId: 22372 
Id: 22397, UpdtId: 22373 
Id: 22398, UpdtId: 22374 
Id: 22400, UpdtId: 22375 
Id: 22402, UpdtId: 22376 
Id: 22403, U

In [28]:
print(len(U_list),len(V_list))
print(U_list[0].shape,V_list[0].shape)
print()
for u,v in zip(U_list[1:],V_list[1:]):
    print(u.shape,v.shape)   

9 9
(22340, 50) (8328, 50)

(22407, 50) (8329, 50)
(22511, 50) (8381, 50)
(22634, 50) (8429, 50)
(22706, 50) (8462, 50)
(22738, 50) (8493, 50)
(22761, 50) (8522, 50)
(22772, 50) (8542, 50)
(22776, 50) (8564, 50)


In [30]:
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [31]:
print(len(In_DomainUSERS),len(In_DomainITEMS))

22776 8564


Data Adjustments

#### Get_newHoldoutDF

In [21]:
newHOLDOUT_LIST = get_NEWHoldout(HOLDOUT_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
print(len(newHOLDOUT_LIST))
print()
for old,new in zip(HOLDOUT_list,newHOLDOUT_LIST):
    print(old.shape[0],new.shape[0])

100%|██████████| 8/8 [00:00<00:00, 164.96it/s]

8

258 178
239 173
253 207
207 165
148 115
97 72
66 44
30 21


In [22]:
newHOLDOUT_LIST[0]['Updated_UserID'].max()

22406

#### Get UserItem MAT

In [23]:
newUserItem_list = adjustedAllDF(UserItemDF_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
print()
for df in newUserItem_list:
    print(df['Updated_UserID'].nunique(),df['Updated_ItemID'].nunique())


22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [24]:
UserItem_Mat = All_SingleStepRatMat(newUserItem_list,'Updated_UserID','Updated_ItemID')
print(len(UserItem_Mat))
UserItem_Mat

8


[<22407x8329 sparse matrix of type '<class 'numpy.float64'>'
 	with 49611 stored elements in Compressed Sparse Row format>,
 <22511x8381 sparse matrix of type '<class 'numpy.float64'>'
 	with 49953 stored elements in Compressed Sparse Row format>,
 <22634x8429 sparse matrix of type '<class 'numpy.float64'>'
 	with 50358 stored elements in Compressed Sparse Row format>,
 <22706x8462 sparse matrix of type '<class 'numpy.float64'>'
 	with 50707 stored elements in Compressed Sparse Row format>,
 <22738x8493 sparse matrix of type '<class 'numpy.float64'>'
 	with 50940 stored elements in Compressed Sparse Row format>,
 <22761x8522 sparse matrix of type '<class 'numpy.float64'>'
 	with 51108 stored elements in Compressed Sparse Row format>,
 <22772x8542 sparse matrix of type '<class 'numpy.float64'>'
 	with 51211 stored elements in Compressed Sparse Row format>,
 <22776x8564 sparse matrix of type '<class 'numpy.float64'>'
 	with 51280 stored elements in Compressed Sparse Row format>]

In [25]:
print(len(U_list),len(V_list))
for u,v in zip(U_list,V_list):
    print(u.shape[0],v.shape[0])   

9 9
22340 8328
22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


In [26]:
print(AllDF_start['userId'].nunique(),AllDF_start['productId'].nunique())
print()
for i,j in zip(AllUpdtUSERS_,AllUpdtITEMS_):
    print(len(i),len(j))

22340 8328

22407 8329
22511 8381
22634 8429
22706 8462
22738 8493
22761 8522
22772 8542
22776 8564


#### Prediction

In [77]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [27]:
Vpsi_listUpdt = V_list[1:]
for v in Vpsi_listUpdt:
    print(v.shape)

(8329, 50)
(8381, 50)
(8429, 50)
(8462, 50)
(8493, 50)
(8522, 50)
(8542, 50)
(8564, 50)


In [28]:
V_1 = Vpsi_listUpdt[0]
V_1.shape

(8329, 50)

In [29]:
Top_NPred = TopNPred(UserItem_Mat[0],newHOLDOUT_LIST[0],V_1,'Updated_UserID',10)
Top_NPred

array([[5977, 1409,   85, ..., 5824, 1813,   80],
       [  85,  297, 1260, ...,   41,  520,   63],
       [ 297,  519,   80, ..., 1202,  520, 2255],
       ...,
       [1566, 2031,  538, ..., 1375, 2513, 3030],
       [ 311,  297,   85, ...,   83,  381,  613],
       [  67, 5977, 3666, ...,  133,  606,   22]])

In [30]:
Hitrate_Eval(newHOLDOUT_LIST[0],Top_NPred,'Updated_UserID','Updated_ItemID')

Number of hits:  55
Total Num of users:  178
Recommendation HitRate:  0.3089887640449438


0.3089887640449438

In [31]:
AllUpdt_pred =  TQDMgetALLTopNPred(UserItem_Mat,newHOLDOUT_LIST,Vpsi_listUpdt,'Updated_UserID',N = 10)
print(len(AllUpdt_pred))
AllUpdt_pred[:2]

8it [00:00, 34.18it/s]

8


[array([[5977, 1409,   85, ..., 5824, 1813,   80],
        [  85,  297, 1260, ...,   41,  520,   63],
        [ 297,  519,   80, ..., 1202,  520, 2255],
        ...,
        [1566, 2031,  538, ..., 1375, 2513, 3030],
        [ 311,  297,   85, ...,   83,  381,  613],
        [  67, 5977, 3666, ...,  133,  606,   22]]),
 array([[ 439,  842,  230, ..., 3985,  415, 1230],
        [ 439,  842,  230, ...,  800,  415, 1230],
        [ 389,  133,   50, ...,   63,   80,  519],
        ...,
        [1325,  457,   18, ...,  395,  655, 1512],
        [  74,  231,  137, ..., 8034,  555,  519],
        [ 995,  692, 4017, ..., 4722, 3293, 2242]])]

In [33]:
AllSteps_HitR_updt, LowerBand_updt, Avg_HitR_updt, UpperBand_updt   = getAll_HitRate(newHOLDOUT_LIST,AllUpdt_pred,'Updated_UserID','Updated_ItemID')

Number of hits:  55
Total Num of users:  178
Recommendation HitRate:  0.3089887640449438
Number of hits:  53
Total Num of users:  173
Recommendation HitRate:  0.3063583815028902
Number of hits:  82
Total Num of users:  207
Recommendation HitRate:  0.3961352657004831
Number of hits:  45
Total Num of users:  165
Recommendation HitRate:  0.2727272727272727
Number of hits:  9
Total Num of users:  115
Recommendation HitRate:  0.0782608695652174
Number of hits:  4
Total Num of users:  72
Recommendation HitRate:  0.05555555555555555
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.17725326363704533


### Regular PSI 

#### Get Rating Matrices

In [34]:
print(AllDF_list[-1]['userId'].nunique(),AllDF_list[-1]['productId'].nunique())
print(AMZB_DF['userId'].nunique(),AMZB_DF['productId'].nunique())

22799 8620
22799 8620


In [35]:
rows_i = AMZB_DF['userId'].nunique()
cols_i = AMZB_DF['productId'].nunique()
print(A0_df.shape)
A0_Rating_matrix = SingleRatingMatrix(A0_df,'userId', 'productId',rows_i,cols_i)
A0_Rating_matrix

(45139, 4)


<22799x8620 sparse matrix of type '<class 'numpy.float64'>'
	with 44740 stored elements in Compressed Sparse Row format>

In [36]:
new_PSIDFlist = adjustedPSI_DF(PSITest_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
for old,new in zip(PSITest_list,new_PSIDFlist):
    print(old.shape[0],new.shape[0])

110 89
104 96
178 171
114 106
25 21
18 14
10 9
3 3


In [37]:
print("Length of PSI_Train :",len(ΔA_train))
print("Length of PSI_Test  :",len(new_PSIDFlist))

Length of PSI_Train : 10
Length of PSI_Test  : 8


In [38]:
PSI_train_matrix = AllRatingMatrices(ΔA_train,'userId','productId',rows_i ,cols_i)
PSI_train_matrix   

[<22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 553 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 518 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 432 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 397 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 1017 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 482 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 357 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse mat

In [39]:
PSI_test_matrix = AllRatingMatrices(new_PSIDFlist,'Updated_UserID','Updated_ItemID',rows_i ,cols_i)                                                                      
print(len(PSI_test_matrix))
PSI_test_matrix


8


[<22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 88 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 96 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 171 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 106 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 21 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 14 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 9 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 3 stored elements in Compressed Sparse Row format>]

In [40]:
UserItemMAT_list = AllRatingMatrices(newUserItem_list,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
UserItemMAT_list  

[<22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 49611 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 49953 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 50358 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 50707 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 50940 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 51108 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 51211 stored elements in Compressed Sparse Row format>,
 <22799x8620 sparse matrix of type '<class 'numpy.float64'>'
 	with 51280 stored elements in Compressed Sparse Row format>]

#### PSI Prediction

In [41]:
                                                                                   #integratorOnMat(A0,ΔA_train_matrix,ΔA_test_matrix,k):
Vpsi_listReg =  integratorOnMat(A0_Rating_matrix,PSI_train_matrix,PSI_test_matrix,k=50)  #PSI == projector splitting integrator
print(len(Vpsi_listReg))  

8


In [42]:
Top10Pred_psi = TopNPred(UserItemMAT_list[0], newHOLDOUT_LIST[0],Vpsi_listReg[0],'Updated_UserID',N=10)
print(Top10Pred_psi.shape)
Top10Pred_psi

(178, 10)


array([[ 519,  381,   63, ...,  922,   83,   85],
       [  63,  520,   43, ...,  137,   36,  134],
       [ 519,   80,   41, ...,  667, 5977, 1725],
       ...,
       [1566,  538, 2031, ..., 1375, 2513, 3030],
       [ 311,   85,  191, ...,  613, 1725, 1049],
       [  67,   83,  416, ..., 1392,  393,  389]])

In [43]:
Hitrate_Eval(newHOLDOUT_LIST[0],Top10Pred_psi,'Updated_UserID','Updated_ItemID')

Number of hits:  54
Total Num of users:  178
Recommendation HitRate:  0.30337078651685395


0.30337078651685395

In [44]:
All_PSI_PRED =  TQDMgetALLTopNPred(UserItemMAT_list,newHOLDOUT_LIST,Vpsi_listReg,'Updated_UserID',N = 10)
print(len(All_PSI_PRED))
All_PSI_PRED[:2]

8it [00:00, 33.21it/s]

8


[array([[ 519,  381,   63, ...,  922,   83,   85],
        [  63,  520,   43, ...,  137,   36,  134],
        [ 519,   80,   41, ...,  667, 5977, 1725],
        ...,
        [1566,  538, 2031, ..., 1375, 2513, 3030],
        [ 311,   85,  191, ...,  613, 1725, 1049],
        [  67,   83,  416, ..., 1392,  393,  389]]),
 array([[ 297, 1230,  800, ...,  558,  922,  319],
        [ 297, 1230,  800, ...,  558,  319,  922],
        [ 389,  133,  598, ..., 1034,   82, 1310],
        ...,
        [  69,   68, 1512, ...,   85, 2199,   83],
        [  74,  231,  137, ...,   68, 1512,  105],
        [ 995,  717,  692, ..., 4017, 3293,  916]])]

In [45]:
AllSteps_Hitrate_psi, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHOLDOUT_LIST,All_PSI_PRED,'Updated_UserID','Updated_ItemID')

Number of hits:  54
Total Num of users:  178
Recommendation HitRate:  0.30337078651685395
Number of hits:  55
Total Num of users:  173
Recommendation HitRate:  0.3179190751445087
Number of hits:  81
Total Num of users:  207
Recommendation HitRate:  0.391304347826087
Number of hits:  45
Total Num of users:  165
Recommendation HitRate:  0.2727272727272727
Number of hits:  10
Total Num of users:  115
Recommendation HitRate:  0.08695652173913043
Number of hits:  3
Total Num of users:  72
Recommendation HitRate:  0.041666666666666664
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.17674308382756493


###  RandRec:Test

In [53]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


In [46]:
All_RandPred = get_ALLRandPred(UserItem_Mat,newHOLDOUT_LIST,'Updated_UserID',N=10)
All_RandPred[:2]  

8it [00:00, 33.68it/s]


[array([[ 916,  135, 2369, ..., 2396, 7568, 1164],
        [5259,  948, 3558, ..., 6044, 7425, 3776],
        [3535, 7126, 2416, ..., 7303,  494,  660],
        ...,
        [5370, 2396, 7026, ..., 3765, 6754, 7275],
        [7092, 7275, 2504, ..., 1177, 5052,  313],
        [7459, 7939, 5155, ..., 2826, 6447, 7181]]),
 array([[4576, 7522, 1807, ..., 7767, 6350, 2145],
        [4292, 2130, 6669, ..., 1967, 2072, 3453],
        [4264, 3162, 1123, ..., 1179, 4896, 1963],
        ...,
        [4374, 2839, 5222, ..., 4700, 8058, 4098],
        [5078, 8186, 1460, ..., 2931, 4394, 3164],
        [3873, 5482, 7166, ..., 1040, 2558, 7320]])]

In [47]:
AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand = getAll_RandomHitRate(newHOLDOUT_LIST,All_RandPred,'Updated_UserID','Updated_ItemID')

Number of hits:  0
Total Num of users:  178
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  173
Recommendation HitRate:  0.0
Number of hits:  1
Total Num of users:  207
Recommendation HitRate:  0.004830917874396135
Number of hits:  0
Total Num of users:  165
Recommendation HitRate:  0.0
Number of hits:  1
Total Num of users:  115
Recommendation HitRate:  0.008695652173913044
Number of hits:  0
Total Num of users:  72
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  44
Recommendation HitRate:  0.0
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.0016908212560386472


### Most Pop Rec

In [48]:
new_PSIDFlist = adjustedPSI_DF(PSITest_list,userID_dict,itemID_dict,AllUpdtUSERS_,AllUpdtITEMS_,n=8)
for old,new in zip(PSITest_list,new_PSIDFlist):
    print(old.shape[0],new.shape[0])

110 89
104 96
178 171
114 106
25 21
18 14
10 9
3 3


In [ ]:
# MostPOPPred_ = getMOSTPOP_Pred(new_PSIDFlist[0],'Updated_UserID','Updated_ItemID',Nusers=10,N=10)
# MostPOPPred_

In [51]:
All_MostPOPRED_List =  getAll_MOSTPOP_Pred(new_PSIDFlist,newHOLDOUT_LIST,'Updated_UserID','Updated_ItemID',N=10)
All_MostPOPRED_List[:2]

[array([[ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        ...,
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106],
        [ 74, 251, 168, ..., 230, 555, 106]]),
 array([[110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        ...,
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67],
        [110, 319,  74, ..., 239, 251,  67]])]

In [52]:
AllSteps_Hitrate, LowerBand, Avg_HitRate, UpperBand  = getAll_MostPOPHitRate(newHOLDOUT_LIST,All_MostPOPRED_List,'Updated_UserID','Updated_ItemID')

Number of hits:  22
Total Num of users:  178
Recommendation HitRate:  0.12359550561797752
Number of hits:  23
Total Num of users:  173
Recommendation HitRate:  0.1329479768786127
Number of hits:  57
Total Num of users:  207
Recommendation HitRate:  0.2753623188405797
Number of hits:  32
Total Num of users:  165
Recommendation HitRate:  0.19393939393939394
Number of hits:  18
Total Num of users:  115
Recommendation HitRate:  0.1565217391304348
Number of hits:  1
Total Num of users:  72
Recommendation HitRate:  0.013888888888888888
Number of hits:  4
Total Num of users:  44
Recommendation HitRate:  0.09090909090909091
Number of hits:  0
Total Num of users:  21
Recommendation HitRate:  0.0
Average HitRate for All Recommendations:  0.12339561427562232


In [53]:
print(len(UserItem_Mat),len(newHOLDOUT_LIST))

8 8


## Apply Functions on *AMZGames*

### Import Cleaned Data

In [ ]:
AMZGames_DF  = pd.read_csv('/content/usedAMZGames.csv')
AMZGames_DF['timestamp'] = pd.to_datetime(AMZGames_DF['timestamp']) 
print(AMZGames_DF.shape)       #productId_int	userId_int
print(AMZGames_DF.dtypes)
print()
AMZGames_DF

(372453, 6)
userId_int               object
productId_int            object
rating                    int64
timestamp        datetime64[ns]
userId                    int64
productId                 int64
dtype: object



,userId_int,productId_int,rating,timestamp,userId,productId
0,A3BEJOL2X8PS8I,B00004TC6E,1,2001-01-01,0,0
1,A1SQUI6BNEOB6C,B00004UE0G,1,2001-01-01,1,1
2,A2ZESFCRJL7YA0,B00002CF9M,1,2001-01-01,2,2
3,A3URVWPFJOHT72,B00000K1AF,1,2001-01-01,3,3
4,A1EE8EZLR8LJ9M,B00004WLMA,1,2001-01-02,4,4
...,...,...,...,...,...,...
372448,A2UVBD3N7449IC,B01ABTJ2BM,1,2018-08-31,28129,16648
372449,A27F1N27M1X0NM,B01D8H09LQ,1,2018-08-31,3183,16542
372450,A1J5HIF41ENSMZ,B01D8H09LQ,1,2018-08-31,16433,16542
372451,A18C19UTTAQEOD,B01GW3P9PE,1,2018-08-31,16400,17342


In [ ]:
AMZGames_DF = AMZGames_DF[['userId','productId','rating','timestamp']]
print(AMZGames_DF.shape)
print("For users: ")
print(AMZGames_DF['userId'].nunique())
print(AMZGames_DF['userId'].max())
print()
print("For items: ")
print(AMZGames_DF['productId'].nunique())
print(AMZGames_DF['productId'].max())
print()
print("Timestamp")
print(AMZGames_DF['timestamp'].min())
print(AMZGames_DF['timestamp'].max())
AMZGames_DF

(372453, 4)
For users: 
53242
53241

For items: 
17351
17350

Timestamp
2001-01-01 00:00:00
2018-08-31 00:00:00


,userId,productId,rating,timestamp
0,0,0,1,2001-01-01
1,1,1,1,2001-01-01
2,2,2,1,2001-01-01
3,3,3,1,2001-01-01
4,4,4,1,2001-01-02
...,...,...,...,...
372448,28129,16648,1,2018-08-31
372449,3183,16542,1,2018-08-31
372450,16433,16542,1,2018-08-31
372451,16400,17342,1,2018-08-31


###Step 1: Data Splitting

In [ ]:
pivotMonths_list  = getPivotMonths(AMZGames_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-07-31 00:00:00'),
 Timestamp('2018-06-30 00:00:00'),
 Timestamp('2018-05-31 00:00:00'),
 Timestamp('2018-04-30 00:00:00'),
 Timestamp('2018-03-31 00:00:00'),
 Timestamp('2018-02-28 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-09-30 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-02-28 00:00:00')]

In [ ]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZGames_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-02-27 00:00:00
Num ΔA_train : 10


In [ ]:
AllDF_list, PSITest_list,HOLDOUT_list = TestTrain_DataSplit(AMZGames_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [ ]:
for df in ΔA_train:
    print(df['userId'].nunique(),df['productId'].nunique())

1959 2319
1455 1844
1236 1637
1080 1492
1033 1490
1041 1577
907 1355
871 1203
862 1228
845 1238


In [ ]:
for df in ΔA_test:
    print(df['userId'].nunique(),df['productId'].nunique())

865 1224
687 1080
816 1462
718 1183
560 901
465 584
405 421
243 256


## Apply Functions on *AMZElect*

### Import Cleaned Data

In [ ]:
!unzip  /content/usedAMZElect.zip

Archive:  /content/usedAMZElect.zip
  inflating: usedAMZElect.csv        


In [ ]:
AMZElect_DF  = pd.read_csv('/content/usedAMZElect.csv')
AMZElect_DF['timestamp'] = pd.to_datetime(AMZElect_DF['timestamp']) 
print(AMZElect_DF.shape)      #productId_int	userId_int
print(AMZElect_DF.dtypes)
AMZElect_DF

(2760569, 4)
userId                int64
productId             int64
rating                int64
timestamp    datetime64[ns]
dtype: object


,userId,productId,rating,timestamp
0,0,0,1,2000-01-01
1,1,1,1,2000-01-02
2,2,2,1,2000-01-03
3,3,3,1,2000-01-06
4,4,4,1,2000-01-06
...,...,...,...,...
2760564,60863,220253,1,2018-08-31
2760565,9978,220253,1,2018-08-31
2760566,59037,238414,1,2018-08-31
2760567,75909,251125,1,2018-08-31


In [ ]:
print(AMZElect_DF.shape)
print("For users: ")
print(AMZElect_DF['userId'].nunique())
print(AMZElect_DF['userId'].max())
print()
print("For items: ")
print(AMZElect_DF['productId'].nunique())
print(AMZElect_DF['productId'].max())
print()
print("Timestamp")
print(AMZElect_DF['timestamp'].min())
print(AMZElect_DF['timestamp'].max())
AMZElect_DF

(2760569, 4)
For users: 
161594
161593

For items: 
318020
318019

Timestamp
2000-01-01 00:00:00
2018-08-31 00:00:00


,userId,productId,rating,timestamp
0,0,0,1,2000-01-01
1,1,1,1,2000-01-02
2,2,2,1,2000-01-03
3,3,3,1,2000-01-06
4,4,4,1,2000-01-06
...,...,...,...,...
2760564,60863,220253,1,2018-08-31
2760565,9978,220253,1,2018-08-31
2760566,59037,238414,1,2018-08-31
2760567,75909,251125,1,2018-08-31


###Step 1: Data Splitting

In [ ]:
pivotMonths_list  = getPivotMonths(AMZElect_DF,'timestamp',N_TMonths=18)  
pivotMonths_list

[Timestamp('2018-07-31 00:00:00'),
 Timestamp('2018-06-30 00:00:00'),
 Timestamp('2018-05-31 00:00:00'),
 Timestamp('2018-04-30 00:00:00'),
 Timestamp('2018-03-31 00:00:00'),
 Timestamp('2018-02-28 00:00:00'),
 Timestamp('2018-01-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-09-30 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-05-31 00:00:00'),
 Timestamp('2017-04-30 00:00:00'),
 Timestamp('2017-03-31 00:00:00'),
 Timestamp('2017-02-28 00:00:00')]

In [ ]:
A0_df,ΔA_train,ΔA_test = Time_DataSplit(AMZElect_DF,'timestamp',pivotMonths_list,N_TMonths= 18,n_train=10)
print('Last date in A0_df :',A0_df['timestamp'].max())
print('Num ΔA_train :', len(ΔA_train))

Last date in A0_df : 2017-02-27 00:00:00
Num ΔA_train : 10


In [ ]:
AllDF_list, PSITest_list,HOLDOUT_list = TestTrain_DataSplit(AMZElect_DF,'userId','timestamp',pivotMonths_list,ΔA_test)
print('Num AllDF_list :', len(AllDF_list))

Num AllDF_list : 8


In [ ]:
for df in ΔA_train:
    print(df['userId'].nunique(),df['productId'].nunique())

17570 21425
15010 19188
13933 17058
12711 16008
12667 15464
12695 15398
11746 14523
10733 13206
10129 12593
10375 12375


In [ ]:
for df in ΔA_test:
    print(df['userId'].nunique(),df['productId'].nunique())

11572 13325
9614 11772
10842 13592
9888 12416
8939 10901
7677 8770
6609 6753
4279 3895


In [ ]:
print(A0_df['productId'].max(),A0_df['productId'].nunique())
print(A0_df['userId'].max(),A0_df['userId'].nunique())

293278 293279
158466 158467
